In [ ]:
import os
import cv2
import numpy as np
import trimesh
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.utils import save_image, make_grid
from utils.people_snapshot_dataset import People_Snapshot_Dataset
from utils.multi_garment_dataset import Multi_Garment_Dataset
from utils.iper_dataset import iPER_Dataset
from models.networks.smpl import SMPL
from models.networks.render import SMPLRenderer
from utils.util import load_obj, load_pickle_file, write_pickle_file

In [ ]:
data_root = 'data/Multi-Garment_dataset'
pose_cam_path = 'assets/pose_cam.pkl'
isHres = True
image_size = 256
batch_size = 16
num_frame = 1

In [ ]:
smpl = SMPL(pkl_path='assets/smpl_model.pkl', isHres=isHres).cuda()
if isHres:
    faces = smpl.faces_hres
else:
    faces = smpl.faces
smpl_render = SMPLRenderer(faces=faces).cuda()

In [ ]:
smpl_render.set_bg_color((1, 1, 1))

In [ ]:
train_dataset = Multi_Garment_Dataset(data_root=data_root, pose_cam_path=pose_cam_path, num_frame=num_frame, isHres=isHres)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for i, data in enumerate(train_loader):
    
    shape_src = data['shape'].cuda()
    v_personal = data['v_personal'].cuda()
    uv_image = data['uv_image'].cuda()
    f2vts = data['f2vts'].cuda()
    
    tex = smpl_render.extract_tex(uv_image, smpl_render.points_to_sampler(f2vts))
    
    print('shape: ', shape.size())
    print('v_personal: ', v_personal.size())
    print('uv_image: ', uv_image.size())
    print('f2vts: ', f2vts.size())
    print('tex: ', tex.size())
    
    if i >= 0:
        break

In [ ]:
data_ref_root = 'data/iPER'
train_ID_list = []
for line in open(os.path.join(source_data_root, 'train.txt')):
    train_ID_list.append(line.split()[0])
print(len(train_ID_list))

In [ ]:
video_ID = train_ID_list[1]
print(video_ID)
imgs_path = os.path.join(data_ref_root, 'images', video_ID)
pose_shape_pkl_path = os.path.join(data_ref_root, 'smpls', video_ID, 'pose_shape.pkl')
train_ref_dataset = iPER_Dataset(imgs_path, pose_shape_pkl_path, image_size=image_size)
train_ref_loader = DataLoader(train_ref_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for i, data in enumerate(train_ref_loader):
    img_ref = data['image'].cuda()
    shape_ref = data['shape'].cuda()
    pose_ref = data['pose'].cuda()
    cam_ref = data['cam'].cuda()
    print('img: ', img_ref.size())
    print('shape: ', shape_ref.size())
    print('pose: ', pose_ref.size())
    print('cam: ', cam_ref.size())
    if i >= 0:
        break

In [ ]:
verts = smpl(shape, pose, v_personal)
verts = smpl_render.project_to_image(verts, cam, flip=True, withz=True)

In [ ]:
img_masked = smpl_render.render(verts, tex)
print(img_masked.size())

In [ ]:
smpl_render.set_ambient_light(int_dir=0.3, int_amb=0.7, direction=(1, 0.5, 0.5))

In [ ]:
img_smpl = smpl_render.render(verts)
print(img_smpl.size())

In [ ]:
vis_ID = 3

In [ ]:
img_vis = (img.detach().cpu().numpy()[vis_ID] * 255).astype(np.uint8).transpose(1, 2, 0)
plt.imshow(img_vis)

In [ ]:
img_masked_vis = (img_masked.detach().cpu().numpy()[vis_ID] * 255).astype(np.uint8).transpose(1, 2, 0)
plt.imshow(img_masked_vis)

In [ ]:
img_smpl_vis = (img_smpl.detach().cpu().numpy()[vis_ID] * 255).astype(np.uint8).transpose(1, 2, 0)
plt.imshow(img_smpl_vis)